In [1]:
from chapter1 import *

## Finding ground states

Having found a way of encoding the states, the next step is to implement a way of finding the ground state. To this end, we consider a nearest-neighbour Hamiltonian H, of the form
$$H = \sum_n h_{n, n+1}.$$
Here $h_{n,n+1}$ is a hermitian operator acting non-trivially on the sites $n$ and $n+1$. As in any variational approach, the variational principle serves as a guide for finding ground-state approximations, we want to minimise the expectation value of the energy,
$$ \min_A \frac{<\Psi(A)| H | \Psi(A) >}{<\Psi(A)|\Psi(A)>}. $$

In the thermodynamic limit the energy diverges with system size, but, since we are working with translation-invariant states only, we should rather minimise the energy density. We also will restrict to properly normalised states. Diagrammatically, the minimization problem is recast as

![minDiagram](img/2minham.png)

We now turn to some numerical optimization strategies for minimizing this energy density directly.

### The gradient

Any optimization problem relies on an efficient evaluation of the gradient, so the first thing to do is to compute this quantity (efficiently). The objective function $f$ that we want to minimize is a real function of the complex-valued $A$, or equivalently, the independent variables $A$ and $Abar$. The gradient $g$ is then obtained by differentiating $f(Abar,A)$ with respect to Abar,

![gradCalc](img/gradientCalc.png)

If we make sure that the MPS is properly normalised, and subtract the current energy density from every term in the hamiltonian, the gradient takes on the simple form
$$ g = 2 \partial_Abar <\Psi(A)| h | \Psi(A) >.$$
It will prove useful to implement 

The gradient is obtained by differentiating the expression

![grad2](img/grad2.png)

with respect to $Abar$. Differentiating with respect to one $Abar$ tensor amounts to leaving out that tensor, and interpreting the open legs as outgoing ones, i.e. each term looks like

![gradTerms](img/gradTerms.png)

In [2]:
def reducedHamUniform(h, A, l=None, r=None):
    """
    Renormalise Hamiltonian such that its expectation value is 0.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian that needs to be reduced,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array(D, D), optional
            left fixed point of transfermatrix,
            normalised.
        r : np.array(D, D), optional
            right fixed point of transfermatrix,
            normalised.
            
        Returns
        -------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight.
    """
    
    # calculate fixed points if not supplied
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate expectation value
    e = expVal2Uniform(h, A, l, r)
    
    # substract from hamiltonian
    hTilde = h - e * ncon((np.eye(d), np.eye(d)), ([-1, -3], [-2, -4]))
    
    return hTilde

#### Terms of the 'center' kind
The first kind of terms that arise is are obtained by removing one of the $Abar$ on the legs of the Hamiltonian term. This leads to

![gradTerms](img/centerTerms.png)

In [3]:
def gradCenterTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the center terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array(D, D), optional
            left fixed point of transfermatrix,
            normalised.
        r : np.array(D, D), optional
            right fixed point of transfermatrix,
            normalised.
        
        Returns
        -------
        term1 : np.array(D, d, D)
            first term of gradient,
            ordered left-mid-right.
        term2 : np.array(D, d, D)
            second term of gradient,
            ordered left-mid-right.
    """
    
    # calculate fixed points if not supplied
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate first contraction
    term1 = ncon((l, r, A, A, np.conj(A), hTilde), ([6, 1], [5, -3], [1, 3, 2], [2, 4, 5], [6, 7, -1], [3, 4, 7, -2]))
    
    # calculate second contraction
    term2 = ncon((l, r, A, A, np.conj(A), hTilde), ([-1, 1], [5, 7], [1, 3, 2], [2, 4, 5], [-3, 6, 7], [3, 4, -2, 6]))
    
    return term1, term2

#### Terms of the 'left' kind
For the terms where we omit an $Abar$ tensor to the left of the operator $h$, we can contract everything to the left of this missing $Abar$ tensor to the fixed point $l$, and everything to the right of the site containing the Hamiltonian is $r$.

In between these two parts of the network, there is $E^n$, the transfermatrix multiplied $n$ times where $n$ is the separation between the two regions. Thus, summing all terms of the 'left' kind together means that we sum over $n$, and the relevant tensor becomes

$$Esum = 1 + E + E^2 + \dots = \frac{1}{1-E}.$$

However, we must be careful when doing this, as the transfer matrix has leading eigenvalue 1, this quantity will diverge. This can be solved by defining a regularized transfer matrix $Etilde$, substracting the divergent part:

![regTransfer](img/regTransfer.png)

and only then taking the inverse. Note that this will not change the result, as we are working with a regularised hamiltonian such that the contributions we substracted would have evaluated to 0.

$$ Esumtilde = \frac{1}{1-Etilde} =: (1 - E)^p $$

Using this, we define the partial contraction

![Rh](img/Rh.png)

such that the sum of all left terms equals

![leftTerms](img/leftTerm.png)

Concretely, implementing this inverse naively would be an ill-defined problem, so we resort to other algorithms to find $R_h$. Multiplying both sides with $(1-Etilde)$ results in an equation of the form $Ax = b$, which may be solved for $x$ by implementing a  Generalized Minimal RESidual (GMRES) algorithm. 

Note that such an algorithm requires only the action of A, not the actual matrix, such that its construction should again be implemented using a handle.

In [4]:
def Rh(hTilde, A, l=None, r=None):
    """
    Find the partial contraction for Rh.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalised.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array(D, D), optional
            left fixed point of transfermatrix,
            normalised.
        r : np.array(D, D), optional
            right fixed point of transfermatrix,
            normalised.
        
        Returns
        -------
        Rh : np.array(D, D)
            result of contraction,
            ordered top-bottom.
    """
    
    D = A.shape[0]
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # construct regularised transferRight operator
    def Etilde(v):
        """
        Applies 1 - Etilde on a right vector.
        """
        v.reshape(D, D)
        
        # transfermatrix contribution
        transfer = ncon((A, np.conj(A), v), ([-1, 2, 1], [-2, 2, 3], [1, 3]))
        
        # fixed point contribution
        fixed = np.trace(v @ l) * r
        
        Etilde = eye(D, D) - transfer + fixed
        
        return np.reshape(Etilde, (D ** 2))
        
       
    
    # construct b
    b = ncon((r, A, A, np.conj(A), np.conj(A), hTilde), ([4, 5], [-1, 2, 1], [1, 3, 4], [-2, 8, 7], [7, 6, 5], [2, 3, 8, 6]))
    
    # solve Ax = b for x
    A = LinearOperator((D ** 2, D ** 2), matvec=Etilde) 
    b.reshape(D ** 2)
    Rh = gmres(A, b)[0]
    
    return Rh.reshape((D, D))

In [5]:
def gradLeftTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the left terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalised.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array(D, D), optional
            left fixed point of transfermatrix,
            normalised.
        r : np.array(D, D), optional
            right fixed point of transfermatrix,
            normalised.
        
        Returns
        -------
        leftTerms : np.array(D, d, D)
            left terms of gradient,
            ordered left-mid-right.
    """
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate partial contraction
    Rh = Rh(hTilde, A, l, r)
    
    # calculate full contraction
    leftTerms = ncon((Rh, A, l), ([1, -3], [2, -2, 1], [-1, 2]))
    
    return leftTerms    

#### Terms of the 'right' kind
In a very similar way, the terms where we leave out an $Abar$ to the right of the operator $h$, can be evaluated with the following contractions:

![Lh](img/Lh.png)

such that the sum of all 'right' terms equals

![rightTerms](img/rightTerm.png)

In [6]:
def Lh(hTilde, A, l=None, r=None):
    """
    Find the partial contraction for Lh.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalised.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array(D, D), optional
            left fixed point of transfermatrix,
            normalised.
        r : np.array(D, D), optional
            right fixed point of transfermatrix,
            normalised.
        
        Returns
        -------
        Lh : np.array(D, D)
            result of contraction,
            ordered bottom-top.
    """
    
    D = A.shape[0]
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # construct regularised transferLeft operator
    def Etilde(v):
        """
        Applies 1 - Etilde on a left vector.
        """
        v.reshape(D, D)
        
        # transfer matrix contribution
        transfer = ncon((v, A, np.conj(A)), ([3, 1], [1, 2, -2], [3, 2, -1]))
        
        # fixed point contribution
        fixed = np.trace(v @ r) * l
        
        Etilde = eye(D, D) - transfer + fixed
        
        return np.reshape(Etilde, (D ** 2))
        
    # construct b
    b = ncon((l, A, A, np.conj(A), np.conj(A), hTilde), ([5, 1], [1, 3, 2], [2, 4, -2], [5, 6, 7], [7, 8, -1], [3, 4, 6, 8]))    
    
    # solve Ax = b for x
    A = LinearOperator((D ** 2, D ** 2), matvec=Etilde) 
    b.reshape(D ** 2)
    Lh = gmres(A, b)[0]
    
    return Lh.reshape((D, D))

In [7]:
def gradRightTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the right terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalised.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array(D, D), optional
            left fixed point of transfermatrix,
            normalised.
        r : np.array(D, D), optional
            right fixed point of transfermatrix,
            normalised.
        
        Returns
        -------
        rightTerms : np.array(D, d, D)
            right terms of gradient,
            ordered left-mid-right.
    """
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate partial contraction
    Lh = Lh(hTilde, A, l, r)
    
    # calculate full contraction
    rightTerms = ncon((Lh, A, r), ([-1, 1], [1, -2, 2], [2, -3]))
    
    return rightTerms

Summing up, the gradient is found by summing up everything:

![gradient](img/gradient.png)

In [8]:
def gradient(h, A, l=None, r=None):
    """
    Calculate the gradient of the expectation value of h @ MPS A.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalised.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array(D, D), optional
            left fixed point of transfermatrix,
            normalised.
        r : np.array(D, D), optional
            right fixed point of transfermatrix,
            normalised.
        
        Returns
        -------
        grad : np.array(D, d, D)
            Gradient,
            ordered left-mid-right.
    """
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
        
    # renormalise Hamiltonian
    hTilde = reducedHamUniform(h, A, l, r)
        
    # find terms
    centerTerm1, centerTerm2 = gradCenterTerms(hTilde, A, l, r)
    leftTerms = gradLeftTerms(hTilde, A, l, r)
    rightTerms = gradRightTerms(hTilde, A, l, r)
    
    grad = 2 * np.sum(centerTerm1, centerTerm2, leftTerms, rightTerad)
    
    return grad

## Gradient descent algorithms
The simplest way to use this information to find the ground state of a Hamiltonian is then to use a method of gradient descent, or just by iterating, for a small step $\epsilon$:

$$ A_{i+1} = A_i - \epsilon g $$

However, this can be further improved upon by using more advanced algorithms, as for example those already implemented by the scipy package.

In [ ]:
def groundStateGradDescent(h, D, eps=1e-3, A0=None, rtol=1e-3):
    """
    Find the ground state using gradient descent.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian to minimise,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        D : int
            Bond dimension
        eps : float
            Stepsize.
        A0 : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            initial guess.
        rtol : float
            Relative convergence criterium.
        
        Returns
        -------
        E : float
            expectation value @ minimum
        A : np.array(D, d, D)
            ground state MPS,
            ordered left-mid-right.
    """
    
    d = h.shape[0]
    
    # if no initial value, choose random
    if A0 is None:
        A0 = createMPS(D, d, D)
    
    # calculate gradient
    g = gradient(h, A0)
    g0 = np.zeros((D, d, D))
    
    A = A0
    
    while not np.allclose(g, g0, rtol):
        # do a step
        A = A - eps * g
        
        # calculate new gradient
        g = gradient(h, A)
        
        # TODO max iteration protection?
    
    # calculate ground state energy
    E = expVal2Uniform(h, A)
    
    return E, A

In [ ]:
def groundStateMinimise(h, D, A0=None, rtol=1e-3):
    """
    Find the ground state using gradient descent.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian to minimise,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        D : int
            Bond dimension
        A0 : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            initial guess.
        rtol : float
            Relative convergence criterium.
        
        Returns
        -------
        E : float
            expectation value @ minimum
        A : np.array(D, d, D)
            ground state MPS,
            ordered left-mid-right.
    """